In [2]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB

Class TSPInstance can be easily modified to tweak the costs for each arc and try different settings. 

The one found now, with 9 vertices, has a standard cost 10 for all edges except for two subsets [(1,2,3) and (4,5,6)] that have negative costs. With no SECS, the optimal path would be a direct edge source (0) to sink (8), and two subtours forming, one for each subset, plus an additional isolated node (7).  

This example shows how our code works with multiple subtours and isolated nodes combined. However, costs can be further personalised or totally eliminated to get a random instance, for deeper studying. 

Along the code there are references to Markdowns, where some parts are explained in more detail

In [9]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    cost: Dict[Arc, float] #contains y coordinates for all our vertices. Vertex i is formed by x[i], y[i]   

    def __init__(self, n, costs:List[float]):
        self.n = n 
        #PERSONALIZED INSTANCE       
        self.cost = {
            (i, j): 10
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }
        self.cost[(1,2)]=-1
        self.cost[(2,3)]=-1
        self.cost[(3,1)]=-1
        self.cost[(4,5)]=-1
        self.cost[(5,6)]=-1
        self.cost[(6,4)]=-1 

        #RANDOM INSTANCE
        """
        Branch-and-Cutself.cost = {}
        cost_index = 0
        for i in self.vertices():
            for j in self.vertices():
                if i != j:
                    self.cost[(i, j)] = costs[cost_index]
                    cost_index += 1
        """

    def vertices(self) -> Iterable[Vertex]: 
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:        
        costs= [uniform(-10,10) for _ in range(n*(n-1))] #RANDOM COSTS IN CASE A RANDOM INSTANCE WANTS TO BE TRIED
        return TSPIntance(n, costs=costs)

In [ ]:
class TSPSolution: 
    tour: Tour 
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [ ]:
class BranchAndCutIntegerSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x') #here we define Xij
        self.__build_model()

    def __build_model(self) -> None:
        self.m.addConstr(self.x.sum(0, '*') == 1) # only one outgoing  arc from source.
        self.m.addConstr(self.x.sum('*', self.tsp.n-1) == 1 ) # only one incoming  arc to sink.
        self.m.addConstrs(self.x.sum(i, '*') == self.x.sum('*',i) for i in range(1, self.tsp.n - 1))  # flow conservation for all nodes
        self.m.addConstr(self.x.sum(self.tsp.n-1, '*') == 0 ) # no outgoing arcs from sink
        self.m.addConstr(self.x.sum('*', 0) == 0 ) # no incoming arcs to source
        self.m.addConstrs((self.x.sum(i, '*') <=1 for i in self.tsp.vertices())) # one outgoing arc at most
        
        
    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1) 
        self.m.optimize(lambda _, where: self.__find_subtours(where=where)) 
        #here we call the optimize gurobi function, that solves the problem.
        #we don't care about the first argument of m.optimize (a Model), so we write '_' in the lambda
        #our second parameter is a number that tells us where we are in the solution function. 
        #we will use it to check when we are at a possible feasible sol, so then we check for subtours. 

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    #here we will look for subtours. Check markdown 1. for further explanation.
    def __find_subtours(self, where: int) -> None: 
        if where != GRB.Callback.MIPSOL: #MIPSOL means gurobi found a possibly feasible solution, which is what we  are looking for now. 
            return
        
        #these are the vertices we are yet to explore. At first we have all of them
        remaining = set(self.tsp.vertices()) 

        while len(remaining) > 0:
            # Get the first vertex of the set
            start = next(iter(remaining))
            subtour = self.__tour_starting_at(start)

            # If subtour is only constituted by the starting node, it means the node was isolated (not a subtour)
            # Alternatively, if the subtour ends in the sink, it means we are dealing with the solution tour.
            # Other cases are considered subtours and therefore a SEC is added on them    
            if subtour != [start] and subtour[-1]!=self.tsp.n-1 :
                self.__add_sec_for(subtour)
                    
            remaining -= set(subtour)

    def __tour_starting_at(self, i) -> Tour: 
        tour = [i] 
        current = self.__next_vertex(i=i)

        #For vertices that are not in optimal sol (isolated vertices), we will find no next vertex.
        if current is None:
            return tour          

        #We keep adding next vertices until we run into the sink, or the vertex we started our tour with.
        # When we stumble upon the starting vertex, we don't include it.      
        while current != i : 
            tour.append(current)
            if current == self.tsp.n-1:
                break
            current = self.__next_vertex(current)

        return tour
    
    #this function moves on to another vertex during our sub_tour search.
    def __next_vertex(self, i: Vertex) -> Vertex:  
        for j in self.tsp.vertices():
            if j == i:
                continue
            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j]) #this is our way to aquire the value of X before reaching the Solution. 
            except GurobiError:
                # When optimisation is over
                x = self.x[i,j].X  # this is the way to reach our value after we reach the optimum. 

            if x > 0.5: #gurobi makes small errors so not all x's will be exactly 1. This is why we write x>0.5 
                return j
            
        return None #Whenever there is no next vertex, we return None (this identifies isolated vertices)
    
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j  in subtour 
                
            ) <= len(subtour)-1
        )

## 1. Subtour detection

To detect subtours, we take a look at the tour that stems from any given node, except for the source or the sink. There are three posibilities:
- The vertex is part of the optimal tour, hence the tour will end at the sink
- The vertex is not part of the optimal solution, which means it is an isolated node.
- The vertex is part of the subtour. In this case the tour will end up in the vertex where it started.

Once this is defined, we check all tours and add SEC's for those that we find to be subtours

In [18]:
tsp = TSPIntance.random(n=9)
solver = BranchAndCutIntegerSolver(tsp=tsp)
solution = solver.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2594094 - for non-commercial use only - registered to al___@bse.eu
Optimize a model with 20 rows, 72 columns and 216 nonzeros
Model fingerprint: 0xfddfafac
Variable types: 0 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 4 rows and 15 columns
Presolve time: 0.00s
Presolved: 16 rows, 57 columns, 163 nonzeros
Variable types: 0 continuous, 57 integer (57 binary)

Root relaxation: objective 4.000000e+00, 0 iterations, 0.00 seconds (0.00 work units)
Adding subtour for [1, 2, 3]
Adding

In [19]:
print(solution)

[0, 8] - Cost: 10.00


In [17]:
import pandas as pd

data = [{"From": i, "To": j, "Cost": cost} for (i, j), cost in tsp.cost.items()]
print(pd.DataFrame(data))


    From  To  Cost
0      0   1    10
1      0   2    10
2      0   3    10
3      0   4    10
4      0   5    10
5      0   6    10
6      0   7    10
7      1   0    10
8      1   2    -1
9      1   3    10
10     1   4    10
11     1   5    10
12     1   6    10
13     1   7    10
14     2   0    10
15     2   1    10
16     2   3    -1
17     2   4    10
18     2   5    10
19     2   6    10
20     2   7    10
21     3   0    10
22     3   1    -1
23     3   2    10
24     3   4    10
25     3   5    10
26     3   6    10
27     3   7    10
28     4   0    10
29     4   1    10
30     4   2    10
31     4   3    10
32     4   5    -1
33     4   6    10
34     4   7    10
35     5   0    10
36     5   1    10
37     5   2    10
38     5   3    10
39     5   4    10
40     5   6    -1
41     5   7    10
42     6   0    10
43     6   1    10
44     6   2    10
45     6   3    10
46     6   4    -1
47     6   5    10
48     6   7    10
49     7   0    10
50     7   1    10
51     7   2